Read table from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [1]:
import pandas as pd

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
l_df = pd.read_html(url)

In [28]:
df = l_df[0]
df = df[df['Borough']!='Not assigned']
df.reset_index(drop=True, inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [32]:
df['Neighbourhood'].value_counts()

Downsview                                                                                                        4
Don Mills                                                                                                        2
Guildwood, Morningside, West Hill                                                                                1
South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens    1
Upper Rouge                                                                                                      1
                                                                                                                ..
Commerce Court, Victoria Hotel                                                                                   1
Cliffside, Cliffcrest, Scarborough Village West                                                                  1
St. James Town                                                                  

The program uses pd.read_html to retrieve table from wiki page
The table is the first dataframe.
by droping the 'bourough' is 'Not assigned', it's the result frame.
By checking value count, no 'Not assigned' in 'neighbourhood' column

In [33]:
df.shape

(103, 3)

-------
#The following is for Q2

In [35]:
!pip install geocoder

     |████████████████████████████████| 98 kB 11.9 MB/s eta 0:00:01


In [36]:
import geocoder

In [37]:
def get_ll(postal_code):
    # initialize variable to None
    lat_lng_coords = None
    
    # loop until get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    return lat_lng_coords[0], lat_lng_coords[1]

In [ ]:
# as geocoder is not stable, the following code is not used
column_names = ['Postal Code', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']
new_df = pd.DataFrame(columns=column_names)

# build new df with latitue and longitude
for i in range(df.shape[0]):
    post_code = df.iloc[i].at['Postal Code']
    borough = df.iloc[i].at['Borough'] 
    neighborhood_name = df.iloc[i].at['Neighbourhood']
    neighborhood_lat, neighborhood_lon = get_ll(post_code)
    
    new_df = new_df.append({'Postal Code': post_code,
                            'Borough': borough,
                            'Neighbourhood': neighborhood_name,
                            'Latitude': neighborhood_lat,
                            'Longitude': neighborhood_lon}, ignore_index=True)

In [44]:
!wget -q -O 'neighbor.csv' https://cocl.us/Geospatial_data


In [45]:
# read data from csv
new_df = pd.read_csv('neighbor.csv')
new_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [46]:
neigh_df = df
neigh_df = neigh_df.join(new_df.set_index('Postal Code'), on='Postal Code')

neigh_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
